In [1]:
import requests
import pandas as pd

legiscan_api_key='c8d73b2173b9881d495a8a05d59d860d'

response = requests.get('https://api.legiscan.com/?key=c8d73b2173b9881d495a8a05d59d860d&op=getDatasetList&state=CA')


{'status': 'OK', 'datasetlist': [{'state_id': 5, 'session_id': 2016, 'year_start': 2023, 'year_end': 2024, 'prefile': 0, 'sine_die': 0, 'prior': 0, 'special': 0, 'session_tag': 'Regular Session', 'session_title': '2023-2024 Regular Session', 'session_name': '2023-2024 Session', 'dataset_date': '2023-06-25', 'dataset_hash': '89d69e51e88506408d80690952568b18', 'dataset_size': 9707689, 'access_key': '4RCd35yT13PDf6j2bWauY6'}, {'state_id': 5, 'session_id': 1791, 'year_start': 2021, 'year_end': 2022, 'prefile': 0, 'sine_die': 1, 'prior': 1, 'special': 0, 'session_tag': 'Regular Session', 'session_title': '2021-2022 Regular Session', 'session_name': '2021-2022 Session', 'dataset_date': '2022-12-11', 'dataset_hash': '48ae3222e08e6cd730ef7c818d467561', 'dataset_size': 22799636, 'access_key': '6x48LfAhHun0gKytklHOTS'}, {'state_id': 5, 'session_id': 1624, 'year_start': 2019, 'year_end': 2020, 'prefile': 0, 'sine_die': 1, 'prior': 1, 'special': 0, 'session_tag': 'Regular Session', 'session_title'

In [4]:
getDatasetList='https://api.legiscan.com/?key=c8d73b2173b9881d495a8a05d59d860d&op=getDatasetList&state=CA'
getDataset='https://api.legiscan.com/?key=c8d73b2173b9881d495a8a05d59d860d&op=getDataset&access_key=4RCd35yT13PDf6j2bWauY6&id=2016'
getSessionList='https://api.legiscan.com/?key=c8d73b2173b9881d495a8a05d59d860d&op=getSessionList&state=CA'
getSessionPeople='https://api.legiscan.com/?key=c8d73b2173b9881d495a8a05d59d860d&op=getSessionPeople&id=2016'
getMonitorListRaw='https://api.legiscan.com/?key=c8d73b2173b9881d495a8a05d59d860d&op=getMonitorListRaw&record=current'
getMonitorList='https://api.legiscan.com/?key=c8d73b2173b9881d495a8a05d59d860d&op=getMonitorList&record=all'
getMasterListRaw='https://api.legiscan.com/?key=c8d73b2173b9881d495a8a05d59d860d&op=getMasterListRaw&id=2016'
getMasterList='https://api.legiscan.com/?key=c8d73b2173b9881d495a8a05d59d860d&op=getMasterList&id=2016'
getBill='https://api.legiscan.com/?key=c8d73b2173b9881d495a8a05d59d860d&op=getBill&id=1773266'
getBillText='https://api.legiscan.com/?key=c8d73b2173b9881d495a8a05d59d860d&op=getBillText&id=2831267'
getRollCall='https://api.legiscan.com/?key=c8d73b2173b9881d495a8a05d59d860d&op=getRollCall&id=1347954'
getPerson='https://api.legiscan.com/?key=c8d73b2173b9881d495a8a05d59d860d&op=getPerson&id=24935'
getSearch='https://api.legiscan.com/?key=c8d73b2173b9881d495a8a05d59d860d&op=getSearch&state=CA&query=tax'
getSearchRaw='https://api.legiscan.com/?key=c8d73b2173b9881d495a8a05d59d860d&op=getSearchRaw&state=CA&query=tax'

In [5]:
df = pd.read_json(getDatasetList)
df.head()

,status,datasetlist
0,OK,"{'state_id': 5, 'session_id': 2016, 'year_star..."
1,OK,"{'state_id': 5, 'session_id': 1791, 'year_star..."
2,OK,"{'state_id': 5, 'session_id': 1624, 'year_star..."
3,OK,"{'state_id': 5, 'session_id': 1400, 'year_star..."
4,OK,"{'state_id': 5, 'session_id': 1120, 'year_star..."
